In [ ]:
import re
import math
import random
import numpy as np
from collections import defaultdict

# ---------------------------
# 1) Corpus pequeño en español
# ---------------------------
corpus = [
    "el rey ama a la reina",
    "la reina ama al rey",
    "el rey gobierna el reino",
    "la reina gobierna el reino",
    "el príncipe es el hijo del rey",
    "la princesa es la hija de la reina",
    "parís es la capital de francia",
    "berlín es la capital de alemania",
    "roma es la capital de italia",
    "madrid es la capital de españa",
    "el dinero está en el banco",
    "deposité dinero en el banco",
    "el banco aprobó el préstamo",
    "trabajo en un banco grande",
    "me senté en un banco del parque",
    "el banco de la plaza es de madera",
    "el banco del parque estaba mojado",
    "los perros y los gatos son animales",
    "un perro es un animal leal",
    "un gato es un animal curioso",
]

# ---------------------------
# Tokenizador simple
# ---------------------------
TOKEN_RE = re.compile(r"[a-záéíóúüñ]+", flags=re.IGNORECASE)

def tokenize(text):
    """Devuelve lista de tokens (minúsculas), mantiene tildes y ñ."""
    return TOKEN_RE.findall(text.lower())

sentences = [tokenize(s) for s in corpus]

# ---------------------------
# Vocabulario
# ---------------------------
vocab = sorted({w for sent in sentences for w in sent})
stoi = {w: i for i, w in enumerate(vocab)}
itos = {i: w for i, w in enumerate(vocab)}

print("Tamaño del vocabulario:", len(vocab))
print("Ejemplo de vocabulario:", vocab[:25])

Tamaño del vocabulario: 53
Ejemplo de vocabulario: ['a', 'al', 'alemania', 'ama', 'animal', 'animales', 'aprobó', 'banco', 'berlín', 'capital', 'curioso', 'de', 'del', 'deposité', 'dinero', 'el', 'en', 'es', 'españa', 'estaba', 'está', 'francia', 'gato', 'gatos', 'gobierna']


In [ ]:
# ---------------------------
# Co-ocurrencias (ventana con ponderación por distancia)
# ---------------------------
window = 4
cooc = defaultdict(float)

for sent in sentences:
    ids = [stoi[w] for w in sent]
    for center_idx, wi in enumerate(ids):
        start = max(0, center_idx - window)
        end   = min(len(ids), center_idx + window + 1)
        for ctx_pos in range(start, end):
            if ctx_pos == center_idx:
                continue
            wj = ids[ctx_pos]
            dist = abs(ctx_pos - center_idx)
            cooc[(wi, wj)] += 1.0 / dist  # palabras más cercanas pesan más

cooc_items = list(cooc.items())
print("Pares de co-ocurrencia:", len(cooc_items))
print("Ejemplo (primer par):", cooc_items[0])


Pares de co-ocurrencia: 373
Ejemplo (primer par): ((15, 46), 2.8333333333333335)


In [ ]:
# ---------------------------
# Hiperparámetros y matrices
# ---------------------------
np.random.seed(0)
random.seed(0)
V = len(vocab)
dim = 30     # dimensión embeddings
lr = 0.05
epochs = 40

x_max = 10.0
alpha = 0.75

W  = np.random.normal(0, 0.1, size=(V, dim))  # word vectors
C  = np.random.normal(0, 0.1, size=(V, dim))  # context vectors
bW = np.zeros(V)
bC = np.zeros(V)

def weight_fn(x):
    """Función de peso de GloVe"""
    return 1.0 if x >= x_max else (x / x_max) ** alpha

In [ ]:
# ---------------------------
# Entrenamiento (SGD)
# ---------------------------
for epoch in range(epochs):
    random.shuffle(cooc_items)
    total_loss = 0.0

    for (i, j), x in cooc_items:
        w = weight_fn(x)
        pred = W[i].dot(C[j]) + bW[i] + bC[j]
        diff = pred - math.log(x)
        total_loss += w * diff * diff

        grad = 2 * w * diff

        wi_old = W[i].copy()

        W[i]  -= lr * grad * C[j]
        C[j]  -= lr * grad * wi_old
        bW[i] -= lr * grad
        bC[j] -= lr * grad

    if (epoch + 1) % 10 == 0:
        print("época", epoch + 1, "| pérdida promedio", total_loss / len(cooc_items))


época 10 | pérdida promedio 0.05475483062432847
época 20 | pérdida promedio 0.02819161635472256
época 30 | pérdida promedio 0.01721151996682706
época 40 | pérdida promedio 0.01049971095522658


In [ ]:

# ---------------------------
# Embedding final y normalización
# ---------------------------
E = W + C
eps = 1e-8
norms = np.linalg.norm(E, axis=1, keepdims=True)
E = E / (norms + eps)

# ---------------------------
# Similitud coseno
# ---------------------------
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def most_similar(word, topn=8):
    if word not in stoi:
        print(f"'{word}' no está en el vocabulario.")
        return

    i = stoi[word]
    query = E[i]

    scores = []
    for j in range(len(E)):
        if j == i:
            continue
        sim = cosine(query, E[j])
        scores.append((sim, j))

    scores.sort(reverse=True)
    for sim, j in scores[:topn]:
        print(f"{itos[j]:15s} {sim:.4f}")

# ---------------------------
# Ejemplos
# ---------------------------
queries = ["rey", "reina", "banco", "dinero", "préstamo", "parque", "capital", "españa", "perro", "gato"]
for w in queries:
    print("\n" + "="*50)
    print("Palabra:", w)
    most_similar(w, topn=8)


Palabra: rey
el              0.6684
al              0.6106
gobierna        0.6100
madera          0.5981
leal            0.5850
banco           0.5843
francia         0.5343
del             0.5302

Palabra: reina
la              0.7672
es              0.7237
plaza           0.6199
capital         0.4931
gato            0.4479
perro           0.4412
deposité        0.4209
de              0.4100

Palabra: banco
el              0.8222
gobierna        0.5963
rey             0.5843
en              0.5076
leal            0.4843
del             0.4713
francia         0.4201
reino           0.4127

Palabra: dinero
roma            0.3930
me              0.3445
deposité        0.3264
hija            0.3184
al              0.2734
mojado          0.2728
perros          0.2453
está            0.2207

Palabra: préstamo
madera          0.3489
senté           0.3119
italia          0.2973
reino           0.2866
alemania        0.2617
mojado          0.2391
madrid          0.2241
berlín          0.191